In [4]:
import numpy as np
from collections import Counter, defaultdict, namedtuple
from math import gcd, ceil
import re
import networkx as nx
from dataclasses import dataclass
from matplotlib import pyplot as plt
# plt.imshow(pic)
from functools    import cache

from aocutils.common import to_int, flatten, reverse_dict, zippify, list_multiply, ints
from aocutils.grid import iterate, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, dimensions, manhattan, positive, conv1d, conv2d
from aocutils.maze import bfs, dijkstra, get_path, dfs
from aocutils.math import lcm, gcd, factors, mul_inv
from aocutils.special import find_pattern_in_iter, find_repeat, find_cycle, md5, CFG, ShuntingYard, binarysearch, deduce_matches
from aocutils.visuals import visualize_graph, labelize, cat, animate_grid

In [5]:
lines = open('input.txt').read().splitlines()
lines = [to_int(l.split()) for l in lines]
lines[0]

IndexError: list index out of range

In [344]:
def hextobin(s):
    l = len(s)
    convert = bin(int(s, 16))[2:]
    prefix = '0' * (len(s) * 4 - len(convert))
    return prefix + convert
assert hextobin('D2FE28') == '110100101111111000101000'
assert hextobin('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'


00


In [378]:
import math  
def parse_bin(binary):
    global ans
    version = int(binary[:3], 2)
    typeid = int(binary[3:6], 2)
    ans += version
    print('received', binary)
    print('version', version, 'type', typeid)
    
    if typeid == 4:
        num = ''
        for i in range(6, len(binary), 5):
            first = binary[i]
            num += binary[i+1:i+5]
            if first != '1':
                # this was the last number
                break
        # print('val', int(num,2))
        return (int(num,2), i+5)
    else:
        # total length in bits
        lentype = binary[6]
        if lentype == '0':
            print('operator packet with lenght')
            fifteen = binary[7:7+15]
            print('total len bits',int(fifteen,2))
            fif = int(fifteen,2)
            pnt = 0
            answers = []
            while pnt < fif:
                res, l = parse_bin(binary[7+15+pnt:7+15+fif])
                print('res received', res, l)
                answers.append(res)
                pnt += l
            pnt = 7+15+fif

        else:
            print('operator packet with packets expected')
            npackets = int(binary[7:7+11],2)
            packets = 0
            pnt = 7+11
            answers = []
            while packets < npackets:
                res, l = parse_bin(binary[pnt:])
                pnt += l
                packets+=1
                answers.append(res)
                
        match typeid:
            case 0:
                res = sum(answers)
            case 1:
                res = math.prod(answers)
            case 2:
                res = min(answers)
            case 3:
                res = max(answers)
            case 5:
                res = 1 if answers[0] > answers[1] else 0
            case 6:
                res = 1 if answers[0] < answers[1] else 0
            case 7:
                res = 1 if answers[0] == answers[1] else 0
        return res, pnt

def parse_hex(s):
    return parse_bin(hextobin(s))
ans = 0
binary = hextobin('620080001611562C8802118E34')
parse_bin(binary)
ans


0
received 01100010000000001000000000000000000101100001000101010110001011001000100000000010000100011000111000110100
version 3 type 0
operator packet with packets expected
received 00000000000000000101100001000101010110001011001000100000000010000100011000111000110100
version 0 type 0
operator packet with lenght
total len bits 22
received 0001000101010110001011
version 0 type 4
res received 10 11
received 10110001011
version 5 type 4
res received 11 11
received 001000100000000010000100011000111000110100
version 1 type 0
operator packet with packets expected
received 000100011000111000110100
version 0 type 4
received 0111000110100
version 3 type 4


12

In [387]:
assert parse_hex('CE00C43D881120')[0]


received 11001110000000001100010000111101100010000001000100100000
version 6 type 3
operator packet with packets expected
received 00010000111101100010000001000100100000
version 0 type 4
received 101100010000001000100100000
version 5 type 4
received 0001000100100000
version 0 type 4


In [388]:
assert parse_hex('C200B40A82')[0] == 3
assert parse_hex('04005AC33890')[0] == 54
assert parse_hex('880086C3E88112')[0] == 7
assert parse_hex('CE00C43D881120')[0] == 9
assert parse_hex('D8005AC2A8F0')[0] == 1
assert parse_hex('F600BC2D8F')[0] == 0
assert parse_hex('9C005AC2F8F0')[0] == 0
assert parse_hex('9C0141080250320F1802104A08')[0] == 1



received 1100001000000000101101000000101010000010
version 6 type 0
operator packet with packets expected
received 1101000000101010000010
version 6 type 4
received 01010000010
version 2 type 4
00000
received 000001000000000001011010110000110011100010010000
version 0 type 1
operator packet with lenght
total len bits 22
received 1011000011001110001001
version 5 type 4
res received 6 11
received 01110001001
version 3 type 4
res received 9 11

received 10001000000000001000011011000011111010001000000100010010
version 4 type 2
operator packet with lenght
total len bits 33
received 101100001111101000100000010001001
version 5 type 4
res received 7 11
received 1101000100000010001001
version 6 type 4
res received 8 11
received 00010001001
version 0 type 4
res received 9 11

received 11001110000000001100010000111101100010000001000100100000
version 6 type 3
operator packet with packets expected
received 00010000111101100010000001000100100000
version 0 type 4
received 101100010000001000100100000
ve

In [391]:
s = 'E058F79802FA00A4C1C496E5C738D860094BDF5F3ED004277DD87BB36C8EA800BDC3891D4AFA212012B64FE21801AB80021712E3CC771006A3E47B8811E4C01900043A1D41686E200DC4B8DB06C001098411C22B30085B2D6B743A6277CF719B28C9EA11AEABB6D200C9E6C6F801F493C7FE13278FFC26467C869BC802839E489C19934D935C984B88460085002F931F7D978740668A8C0139279C00D40401E8D1082318002111CE0F460500BE462F3350CD20AF339A7BB4599DA7B755B9E6B6007D25E87F3D2977543F00016A2DCB029009193D6842A754015CCAF652D6609D2F1EE27B28200C0A4B1DFCC9AC0109F82C4FC17880485E00D4C0010F8D110E118803F0DA1845A932B82E200D41E94AD7977699FED38C0169DD53B986BEE7E00A49A2CE554A73D5A6ED2F64B4804419508B00584019877142180803715224C613009E795E58FA45EA7C04C012D004E7E3FE64C27E3FE64C24FA5D331CFB024E0064DEEB49D0CC401A2004363AC6C8344008641B8351B08010882917E3D1801D2C7CA0124AE32DD3DDE86CF52BBFAAC2420099AC01496269FD65FA583A5A9ECD781A20094CE10A73F5F4EB450200D326D270021A9F8A349F7F897E85A4020CF802F238AEAA8D22D1397BF27A97FD220898600C4926CBAFCD1180087738FD353ECB7FDE94A6FBCAA0C3794875708032D8A1A0084AE378B994AE378B9A8007CD370A6F36C17C9BFCAEF18A73B2028C0A004CBC7D695773FAF1006E52539D2CFD800D24B577E1398C259802D3D23AB00540010A8611260D0002130D23645D3004A6791F22D802931FA4E46B31FA4E4686004A8014805AE0801AC050C38010600580109EC03CC200DD40031F100B166005200898A00690061860072801CE007B001573B5493004248EA553E462EC401A64EE2F6C7E23740094C952AFF031401A95A7192475CACF5E3F988E29627600E724DBA14CBE710C2C4E72302C91D12B0063F2BBFFC6A586A763B89C4DC9A0'
ans = 0
binary = hextobin(s)
parse_bin(binary)[0], ans


received 111000000101100011110111100110000000001011111010000000001010010011000001110001001001011011100101110001110011100011011000011000000000100101001011110111110101111100111110110100000000010000100111011111011101100001111011101100110110110010001110101010000000000010111101110000111000100100011101010010101111101000100001001000000001001010110110010011111110001000011000000000011010101110000000000000100001011100010010111000111100110001110111000100000000011010100011111001000111101110001000000100011110010011000000000110010000000000000100001110100001110101000001011010000110111000100000000011011100010010111000110110110000011011000000000000010000100110000100000100011100001000101011001100000000100001011011001011010110101101110100001110100110001001110111110011110111000110011011001010001100100111101010000100011010111010101011101101101101001000000000110010011110011011000110111110000000000111110100100100111100011111111110000100110010011110001111111111000010011001000110011111001000011010011011110010

(1510977819698, 913)